## Asymmetric Cryptography

More info: https://github.com/alexmgr/tinyec

## Registry
Elliptic-Curve Diffie-Hellman (ECDH) Key Exchange

In [1]:
# Importing necessary modules
from tinyec import registry
import secrets

In [2]:
# Getting the 'brainpoolP256r1' curve from the registry 
curve = registry.get_curve('brainpoolP256r1')

In [3]:
def compress(pubKey):
    return hex(pubKey.x) + hex(pubKey.y % 2)[2:]

### Generating Private Keys

In [4]:
# Generating Alice's private 
alice_privatekey = secrets.randbelow(curve.field.n)
print("Alice's private key: ", alice_privatekey)

print('\n'+'***'*20+'\n')

# Generating Bob's private key
bob_privatekey = secrets.randbelow(curve.field.n)
print("Bob's private key: ", bob_privatekey)

Alice's private key:  2964006587303036042817841926699248083294931531082062889076141140878690715572

************************************************************

Bob's private key:  54894156910797685589199381457378836911646810079898602261753888393942748806016


### Generating Public Keys

In [5]:
# Generate Alice's publickey from her private key and Generator point
alice_publickey = alice_privatekey * curve.g
print("Alice's public key: ", alice_publickey)  # compress(alice_publickey)

print('\n'+'***'*20+'\n')

# Generate Bob's publickey from his private key and Generator point
bob_publickey = bob_privatekey * curve.g
print("Bob's public key: ", bob_publickey)  # compress(bob_publickey)

Alice's public key:  (7965163621592756303380806558376229984089761003754628987605451368705658455217, 12481624616579408749774130452865080783482719539341583632709825475711757359047) on "brainpoolP256r1" => y^2 = x^3 + 56698187605326110043627228396178346077120614539475214109386828188763884139993x + 17577232497321838841075697789794520262950426058923084567046852300633325438902 (mod 76884956397045344220809746629001649093037950200943055203735601445031516197751)

************************************************************

Bob's public key:  (54352677517728492282404728220806477619883274639218258433497452449506164533973, 61638496820423296152786822181297459685034851671348489132478035827581252562182) on "brainpoolP256r1" => y^2 = x^3 + 56698187605326110043627228396178346077120614539475214109386828188763884139993x + 17577232497321838841075697789794520262950426058923084567046852300633325438902 (mod 76884956397045344220809746629001649093037950200943055203735601445031516197751)


### Generate Shared-Keys

In [6]:
# The shared key with Alice
alice_sharedkey = alice_privatekey*bob_publickey
print("Alice's shared secret key: ", alice_sharedkey)  # compress(alice_sharedkey)

print('\n'+'***'*20+'\n')

# The shared key with Bob
bob_sharedkey = bob_privatekey*alice_publickey
print("Bob's shared secret key: ", bob_sharedkey)  # compress(bob_sharedkey)

Alice's shared secret key:  (74356174117108866019583550533269517987952227934698256551320634242071947191031, 43017340892550325796253942788115364051082501472385406145213532433022861242908) on "brainpoolP256r1" => y^2 = x^3 + 56698187605326110043627228396178346077120614539475214109386828188763884139993x + 17577232497321838841075697789794520262950426058923084567046852300633325438902 (mod 76884956397045344220809746629001649093037950200943055203735601445031516197751)

************************************************************

Bob's shared secret key:  (74356174117108866019583550533269517987952227934698256551320634242071947191031, 43017340892550325796253942788115364051082501472385406145213532433022861242908) on "brainpoolP256r1" => y^2 = x^3 + 56698187605326110043627228396178346077120614539475214109386828188763884139993x + 17577232497321838841075697789794520262950426058923084567046852300633325438902 (mod 76884956397045344220809746629001649093037950200943055203735601445031516197751)


### Match Keys
The generated shared secret is a 257-bit integer (compressed EC point for 256-bit curve, encoded as 65 hex digits)

In [7]:
try:
    alice_sharedkey == bob_sharedkey
    print("Shared secret keys match each other!")
except:
    print("Shared secret keys DO NOT match each other.")

Shared secret keys match each other!


### Derive a secret key from the shared key

In [8]:
# symmetric_key_from_ECDH.py
import hashlib

# Convert the x & y components to bytes of length 32
x_component = int.to_bytes(alice_sharedkey.x, 32, 'big')
y_component = int.to_bytes(alice_sharedkey.y, 32, 'big')

# Create a SHA3_256 class
sha3_key = hashlib.sha3_256()

# Update the hash object with x_component
sha3_key.update(x_component)

# Concatenate the y_component with x_component in the hash object
sha3_key.update(y_component)

# Derive the key
secret_key = sha3_key.digest()
print(secret_key)

b":\xc6\xc9\xd1\x16u\x01\xb0H\xdd\r\xaflE\xba\xdd\xd0&c\x11-$\xbb'NR8C]4m|"
